# Simulation Truth

This notebook will introduce you to the concept of simulation truth in fuse.



## Imports and Simulation Context

Similar to the previous notebooks, we will start by importing the necessary modules and creating a simulation context. Additional we register two new plugins called `PeakTruth` and `SurvivingClusters`.

In [25]:
import fuse
import numpy as np

In [26]:
st = fuse.context.full_chain_context(output_folder = "./fuse_data")

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "entry_stop": 10,
               })

run_number = "00000"

Warning! elife not in context config, skipping...
Warning! electron_drift_velocity not in context config, skipping...
Warning! electron_drift_time_gate not in context config, skipping...


/home/hschulze/.local/lib/python3.9/site-packages/strax/context.py:349: UserWarning: Provides of multi-output plugins overlap, deregister old plugins <class 'straxen.plugins.raw_records.daqreader.DAQReader'>.
  warnings.warn(
/home/hschulze/.local/lib/python3.9/site-packages/strax/context.py:349: UserWarning: Provides of multi-output plugins overlap, deregister old plugins <class 'straxen.plugins.raw_records.daqreader.DAQReader'>.
  warnings.warn(
/home/hschulze/.local/lib/python3.9/site-packages/strax/context.py:349: UserWarning: Provides of multi-output plugins overlap, deregister old plugins <class 'straxen.plugins.raw_records.daqreader.DAQReader'>.
  warnings.warn(
/home/hschulze/.local/lib/python3.9/site-packages/strax/context.py:349: UserWarning: Provides of multi-output plugins overlap, deregister old plugins <class 'straxen.plugins.raw_records.daqreader.DAQReader'>.
  warnings.warn(
/home/hschulze/.local/lib/python3.9/site-packages/strax/context.py:349: UserWarning: Provides of

## Raw_Records and Contributing_Clusters

First we will run the simulation up to `raw_records`. The `PMTResponseAndDAQ` plugin now has two outputs, `raw_records` and `contributing_channels`, both are saved to disk when we request fuse to produce `raw_records`.

In [27]:
st.make(run_number, "microphysics_summary")
st.make(run_number, "raw_records")

Now that the data is produced, lets load it. Both are of the same `data_kind` so we can load them together.

In [28]:
raw_records = st.get_array(run_number, ["raw_records", "contributing_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

`contributing_clusters` gives you five additional columns. These are:
- `contributing_clusters` - A list of the clusters that contributed to the `raw_record`
- `s1_photons_per_cluster` - The number of S1 photons that of the corresponding cluster in the `raw_record`
- `s2_photons_per_cluster` - The number of S2 photons that of the corresponding cluster in the `raw_record`
- `ap_photons_per_cluster` - The number of (virtual) PMT afterpulse 'photons'
- `raw_area` - The sum of the contributing photon gains divided by the gain of the PMT

Lets have a look what clusters contributed to the first record:

In [29]:
print(raw_records[0]["contributing_clusters"])

[1 0 0 0 0]


You can see that we get a list of length 5. This is a compromise we need to make as we can't store a list of variable length in a strax. In this case we only store the information of the 5 first clusters that contributed to the record. If there are more than 5 clusters for one record, this information is lost. For simulations with a lot of clusters per event, it makes sense to increase the number of clusters that are stored per record. This can be done by changing the config option `max_contributing_channels_in_truth` of the `PMTResponseAndDAQ` plugin. Lets try this out:

In [30]:
st.set_config({"max_contributing_channels_in_truth": 35,})
st.make(run_number, "raw_records")

In [31]:
raw_records = st.get_array(run_number, ["raw_records", "contributing_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

In [32]:
index = np.argmax(np.sum(raw_records["contributing_clusters"]>0, axis = 1))

print(raw_records[index]["contributing_clusters"])

[133  17  63  20 158  87  98 114   4  10  19  53  97  80 136  45  41  40
  36  35  31 144 147  18 148  16  14  11 156 157 140 113 139  65  66]


Now the list is 35 elements long, just as we requested. Depending on the source this might still be not enough. We can now have a look with how many photons each of these clusters contributed to the record:

In [33]:
print("S1 photons:", raw_records[index]["s1_photons_per_cluster"])
print("S2 photons:", raw_records[index]["s2_photons_per_cluster"])
print("AP photons:", raw_records[index]["ap_photons_per_cluster"])

S1 photons: [8 4 4 3 3 3 3 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
S2 photons: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
AP photons: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


We can have a look at the clusters that contributed to the record. `microphysics_summary` contains the column `cluster_id`. This is the same number as stored in `contributing_clusters`. Please note that `cluster_id` is only unique per chunk of data.

In [34]:
microphysics_summary = st.get_df(run_number, "microphysics_summary")

Loading microphysics_summary: |          | 0.00 % [00:00<?]

In [35]:
microphysics_summary[np.isin(microphysics_summary.cluster_id.values, raw_records[index]["contributing_clusters"])].head()

,e_field,time,endtime,x,y,z,ed,nestid,A,Z,...,x_pri,y_pri,z_pri,cluster_id,xe_density,vol_id,create_S2,photons,electrons,excitons
1,28,2827386342,2827386342,-38.867641,-18.246555,-6.679745,25.684120,8,0,0,...,-44.154747,2.803934,7.933488,11,2.862,1,True,1544,338,246
7,28,2827386343,2827386343,-38.822685,-18.261517,-6.661916,73.418549,8,0,0,...,-44.154747,2.803934,7.933488,4,2.862,1,True,4456,891,842
13,28,2827386343,2827386343,-38.818993,-18.243168,-6.654728,31.419432,8,0,0,...,-44.154747,2.803934,7.933488,10,2.862,1,True,1788,493,344
16,27,2827386343,2827386343,-36.483433,-21.715500,-9.258492,67.145241,8,0,0,...,-44.154747,2.803934,7.933488,14,2.862,1,True,4124,744,774
18,24,2827386343,2827386343,-12.585729,17.032082,-36.443317,257.285248,8,0,0,...,-44.154747,2.803934,7.933488,16,2.862,1,True,13317,5867,2915


## Peaks and peak_truth

Next we can process the simulation result to `peak_basics`. Strax(en) will merge multiple records into a peak. The PeakTruth plugin will evaluate which `raw_records` contribute to a peak and calculate a truth output for each peak. The provided columns for each peak are:
- `s1_photon_number_truth` - The number of S1 photons that contributed to the peak
- `s2_photon_number_truth` - The number of S2 photons that contributed to the peak
- `ap_photon_number_truth` - The number of (virtual) PMT afterpulse 'photons' that contributed to the peak
- `raw_area_truth` - The sum of all contributing photon gains divided by the gains of the PMTs
- `observable_energy_truth` - Estimate of the energy that is associated with the peak.
- `number_of_contributing_clusters` - Number of clusters that contributed to the peak
- `average_x_of_contributing_clusters` - Weighted average of the x position of the clusters that contributed to the peak
- `average_y_of_contributing_clusters` - Weighted average of the y position of the clusters that contributed to the peak
- `average_z_of_contributing_clusters` - Weighted average of the z position of the clusters that contributed to the peak
- `average_x_obs_of_contributing_clusters` - Weighted average of the observed x position of the clusters that contributed to the peak
- `average_y_obs_of_contributing_clusters` - Weighted average of the observed y position of the clusters that contributed to the peak
- `average_z_obs_of_contributing_clusters` - Weighted average of the observed z position of the clusters that contributed to the peak

Lets take a closer look at `observable_energy_truth` using an example: 
If we would have two clusters, the first one with 100 keV energy producig 100 S1 photons and the second one with 10 keV producing 10 S1 photons. After simulation and processing we find two S1 peaks in our data. The first S1 consitis of 90 photons from the first cluster and 5 photons of the second cluster. The `observable_energy_truth` for this peak is calculated as: 90/100 * 100 keV + 5/10 * 10 keV = 90 keV + 5 keV = 95 keV. The second S1 consists of 3 photons from the first cluster and 4 photons of the second cluster. The `observable_energy_truth` for this peak is calculated as: 3/100 * 100 keV + 4/10 * 10 keV = 3 keV + 4 keV = 7 keV. A similar calculation is done for the S2 peaks but replacing the S1 photons with the S2 photons.


In [36]:
st.make(run_number, "peak_truth")
st.make(run_number, "peak_positions")

As strax(en) will take care of the matching of our truth information to the individual peaks, we can simply load the `peak_basics` and `peak_truth` data together.

In [37]:
peak_basics = st.get_df(run_number, ["peak_basics", "peak_truth", "peak_positions"])

Loading plugins: |          | 0.00 % [00:00<?]

For a peak area bias study we could now compare the raw_area to the peak area:

In [38]:
peak_basics[["area", "raw_area_truth"]].head()

,area,raw_area_truth
0,4.869673,4.750000
1,1370.833374,1372.499756
2,61739.500000,61745.238281
3,658.460693,660.759888
4,346802.968750,329564.937500


We might also be interested in the peak classification: 

In [39]:
peak_basics[["type", "s1_photon_number_truth", "s2_photon_number_truth", "ap_photon_number_truth"]].head()

,type,s1_photon_number_truth,s2_photon_number_truth,ap_photon_number_truth
0,0,2,0,0
1,2,0,1061,1
2,1,36270,0,40
3,2,0,483,1
4,2,0,258414,246


Or you might want to check how our position reconstruction is doing: 

In [40]:
peak_basics[["type","x","y", "average_x_obs_of_contributing_clusters", "average_y_obs_of_contributing_clusters", "average_z_obs_of_contributing_clusters"]].head()

,type,x,y,average_x_obs_of_contributing_clusters,average_y_obs_of_contributing_clusters,average_z_obs_of_contributing_clusters
0,0,NaN,NaN,-20.890713,-51.367107,-1.464800
1,2,-20.805504,-51.663246,-20.890713,-51.367107,-1.464800
2,1,-61.009472,10.260664,-19.689362,2.118646,-24.723377
3,2,-39.470741,-18.248409,-38.867641,-18.246555,-6.679745
4,2,-37.180698,-18.869146,-33.300831,-12.801611,-12.879695


## Surviving Clusters
Finally we can evaluate if an energy deposit makes it into a record or a peak. This is done by the `SurvivingClusters` plugin. It will provide the following columns:
- `in_a_record` - Boolean if the cluster is in a record
- `in_a_peak` - Boolean if the cluster is in a peak

In [41]:
st.make(run_number, "surviving_clusters")
microphysics_summary = st.get_df(run_number, ["microphysics_summary", "surviving_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

Now that we have the data loaded we could have a look at clusters that did not make it into a peak: 

In [42]:
microphysics_summary.query("in_a_peak == False").head()

,e_field,time,endtime,x,y,z,ed,nestid,A,Z,...,z_pri,cluster_id,xe_density,vol_id,create_S2,photons,electrons,excitons,in_a_record,in_a_peak
170,27,2827386497,2827386497,32.959923,14.185023,-9.616013,0.096808,0,0,0,...,7.933488,171,2.862,1,True,0,0,0,False,False
203,26,2827386535,2827386535,27.667393,10.273334,-12.147082,0.044494,0,0,0,...,7.933488,204,2.862,1,True,0,0,0,False,False
204,25,2827386598,2827386598,24.774082,14.897573,-23.227947,0.433295,0,0,0,...,7.933488,205,2.862,1,True,1,0,1,False,False
205,25,2827386710,2827386710,9.488111,6.251637,-11.284761,0.141653,0,0,0,...,7.933488,206,2.862,1,True,0,0,0,False,False
207,24,2827386822,2827386822,20.469002,5.807215,-27.482529,0.295280,0,0,0,...,7.933488,208,2.862,1,True,1,0,1,False,False


## Event Truth

Finally we can have a look at truth information at the event level. This is done by the `EventTruth` plugin. It will provide the following columns:
- `x_obs_truth` - The x position of the event. This corresponds to the x position of the main S2.
- `y_obs_truth` - The y position of the event. This corresponds to the y position of the main S2.
- `z_obs_truth` - The z position of the event. This is calculated as mean of the main S1 and S2 `average_z_obs_of_contributing_clusters`. Does this make sense?
- `energy_of_main_peak_truth` - This is intended to be the energy that can be found in the main S1 and S2. It is calculated as the mean of the `observable_energy_truth` of the main S1 and S2. Does this make any sense???
- `total_energy_in_event_truth` - The sum of all energy deposits that are in the event

In [43]:
st.make(run_number, "event_truth")

In [44]:
event_data = st.get_df(run_number, ["event_info", "event_truth"])

Loading plugins: |          | 0.00 % [00:00<?]

First lets take a look at the energy informations: 

In [45]:
event_data[["e_ces", "energy_of_main_peak_truth", "total_energy_in_event_truth"]]

,e_ces,energy_of_main_peak_truth,total_energy_in_event_truth
0,NaN,4.970266,5.101432
1,10214.273438,5828.809570,9707.927734
2,1349.369141,773.168335,1577.445435
3,2639.654053,2207.710205,4019.832275
4,7552.330078,3567.978271,15594.908203
5,16866.316406,737.603027,1264.030273
6,2827.631592,2846.551758,4376.391602
7,1321.256348,1661.187256,1684.907104
8,5.679199,29.439306,31.483326


And the positions: 

In [46]:
event_data[["x", "x_obs_truth", "z", "z_obs_truth"]]

,x,x_obs_truth,z,z_obs_truth
0,NaN,-20.890713,NaN,-1.464800
1,-28.486727,-29.420319,-34.544106,-19.407896
2,-16.824585,-16.463917,-51.321014,-19.869284
3,-51.629799,-47.072384,-19.853037,-14.094154
4,15.486181,0.000000,-17.232950,-12.443692
5,-5.284084,-17.492102,-119.126907,-122.114059
6,-23.652855,-27.306480,-3.039807,-4.481130
7,20.127199,20.087948,-11.545144,-12.389212
8,-50.469139,-50.566116,-0.828222,-0.404144
